# Haystack Multi-Agent Yelp Navigator

Multi-agent supervisor pattern using native Haystack components.

## What You'll Learn

- Multi-agent orchestration using Haystack Pipelines
- `StateMultiplexer` components for feedback loops
- Conditional routing through component outputs
- Supervisor approval patterns in Haystack

## Architecture

**6 Components:** Clarification, Search, Details, Sentiment, Summary, Supervisor

**Key Pattern:** Multiplexers handle many-to-one connections and enable feedback loops from supervisor back to earlier stages.

## Haystack vs LangGraph

| Feature | Haystack | LangGraph |
|---------|----------|-----------|
| **State** | Custom `YelpAgentState` | Inherits `MessagesState` |
| **Routing** | Component outputs | Conditional edges |
| **Feedback** | Multiplexer components | Conditional edges backwards |
| **Execution** | `pipeline.run()` | `graph.stream()` |

## Prerequisites

```bash
cd yelp-navigator
uv run sh build_all_pipelines.sh && sh start_hayhooks.sh
```

Set `OPENAI_API_KEY` in `.env`

In [1]:
import os
from haystack import Pipeline
from haystack_helpers.components import (ClarificationComponent, StateMultiplexer,\
                                        SearchComponent, DetailsComponent, SentimentComponent, SummaryComponent,\
                                        SupervisorComponent)

from dotenv import load_dotenv


# Load environment
load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  WARNING: OPENAI_API_KEY is not set. The LLM components will fail.")

# Hayhooks Configuration
BASE_URL = "http://localhost:1416"


## Imports and Configuration

**Custom Components:** From `haystack_helpers/` - specialized agent components  
**StateMultiplexer:** Critical for feedback loops—merges multiple inputs (initial flow + revision flow) into one output

In [2]:
# ===============================================================================
# 4. BUILDING THE PIPELINE GRAPH
# ===============================================================================

def build_pipeline():
    pipe = Pipeline()

    # 1. Add Components
    pipe.add_component("clarifier", ClarificationComponent())
    
    # Multiplexers (The traffic cops for many-to-one connections)
    pipe.add_component("search_mux", StateMultiplexer("Search Mux"))
    pipe.add_component("details_mux", StateMultiplexer("Details Mux"))
    pipe.add_component("sentiment_mux", StateMultiplexer("Sentiment Mux"))
    pipe.add_component("summary_mux", StateMultiplexer("Summary Mux"))
    
    # Workers
    pipe.add_component("searcher", SearchComponent())
    pipe.add_component("detailer", DetailsComponent())
    pipe.add_component("sentiment", SentimentComponent())
    pipe.add_component("summarizer", SummaryComponent())
    pipe.add_component("supervisor", SupervisorComponent())

    # 2. Connect the "Happy Path" (Left to Right)
    
    # Start -> Clarifier -> Search Mux -> Searcher
    pipe.connect("clarifier.state", "search_mux.source_1")
    pipe.connect("search_mux.state", "searcher.state")
    
    # Searcher -> (Details Mux OR Summary Mux)
    pipe.connect("searcher.to_details", "details_mux.source_1")
    pipe.connect("searcher.to_summary", "summary_mux.source_1")
    
    # Details Mux -> Detailer -> (Sentiment Mux OR Summary Mux)
    pipe.connect("details_mux.state", "detailer.state")
    pipe.connect("detailer.to_sentiment", "sentiment_mux.source_1")
    pipe.connect("detailer.to_summary", "summary_mux.source_2")
    
    # Sentiment Mux -> Sentiment -> Summary Mux
    pipe.connect("sentiment_mux.state", "sentiment.state")
    pipe.connect("sentiment.to_summary", "summary_mux.source_3")
    
    # Summary Mux -> Summarizer -> Supervisor
    pipe.connect("summary_mux.state", "summarizer.state")
    pipe.connect("summarizer.state", "supervisor.state")

    # 3. Connect the "Feedback Loops" (Right to Left / Cycles)
    # Supervisor outputs connect back to the Multiplexers of previous nodes
    
    pipe.connect("supervisor.revise_search", "search_mux.source_2")
    pipe.connect("supervisor.revise_details", "details_mux.source_2")
    pipe.connect("supervisor.revise_sentiment", "sentiment_mux.source_2")
    pipe.connect("supervisor.revise_summary", "summary_mux.source_4")

    return pipe



## Building the Pipeline Graph

Creating a **cyclic pipeline** with conditional routing and feedback loops.

### Key Components

**Multiplexers:** Handle multiple inputs (normal flow + supervisor revisions)  
**Workers:** Perform actual work with multiple outputs for conditional routing  
**Feedback Loops:** Supervisor sends state back to any previous component

### Connection Pattern

```python
# Forward flow
pipe.connect("clarifier.state", "search_mux.source_1")
pipe.connect("search_mux.state", "searcher.state")

# Conditional routing via component outputs
pipe.connect("searcher.to_details", "details_mux.source_1")
pipe.connect("searcher.to_summary", "summary_mux.source_1")

# Feedback loop
pipe.connect("supervisor.revise_search", "search_mux.source_2")
```

In [6]:
# ===============================================================================
# 5. EXECUTION
# ===============================================================================

if __name__ == "__main__":
    pipeline = build_pipeline()
    
    pipeline.draw(path = "haystack_graph.png")
    
    # Draw it (optional, requires graphviz)
    # pipeline.draw("haystack_graph.png")
    
    query = "Find me pizza places in Chicago"
    print(f"🚀 Starting Pipeline with query: {query}\n")
    
    results = pipeline.run(
        {"clarifier": {"query": query}}
    )
    
    final_state = results["supervisor"]["approved"]
    print(f"\n{'='*40}")
    print(f"FINAL RESULT: {final_state.final_summary}")
    print(f"{'='*40}")

🚀 Starting Pipeline with query: Find me pizza places in Chicago


🧠 [Clarification] Processing: Find me pizza places in Chicago
🔀 [Search Mux] Forwarding state...

🔍 [Search] Looking for: pizza places in Chicago
🔀 [Summary Mux] Forwarding state...

📝 [Summary] Generating summary...
👮 [Supervisor] Reviewing (Attempt 1)...
❌ [Supervisor] Needs revision: The summary provides a good starting point, but it could be more comprehensive. It should include a wider variety of pizza types known in Chicago, such as thin crust or tavern-style pizzas, and mention more places to cater to different preferences. Information such as location, price range, or unique features of each place could also be helpful. Additionally, it would be valuable to address the influence of Chicago's pizza scene on local culture or its historical significance. A mention of any notable differences in taste or preparation techniques would enhance the quality and completeness of the summary.. Rerunning search.
🔀 [Search Mux]

![](./images/haystack_graph.png)

## Execution and Visualization

### How It Works

1. Query enters through `clarifier`
2. State propagates through components
3. Components choose outputs based on state (`detail_level`)
4. Supervisor approves or sends back via multiplexer
5. Components track attempts to prevent infinite loops

`pipeline.draw()` visualizes all components, connections, and feedback loops.

## Key Insights

### Advantages

✅ Native Haystack architecture  
✅ Component reusability  
✅ Familiar `Pipeline.run()` API  
✅ Built-in visualization

### Pattern Highlights

**StateMultiplexer for Feedback:**
```python
pipe.connect("clarifier.state", "search_mux.source_1")  # Normal
pipe.connect("supervisor.revise_search", "search_mux.source_2")  # Revision
pipe.connect("search_mux.state", "searcher.state")  # Merged
```

**Conditional Routing:**
```python
pipe.connect("searcher.to_details", "details_mux.source_1")
pipe.connect("searcher.to_summary", "summary_mux.source_1")
```

Both Haystack and LangGraph implement the same supervisor pattern—different paradigms, same result!

**Compare:** See `langgraph_multiagent_supervisor.ipynb` for the LangGraph version